In [26]:
import vectorbt as vbt
import pandas as pd
import numpy as np
from datetime import datetime

In [15]:
btc_price = pd.read_csv("./data/market_data.csv")[["timestamp", "close"]]
# unit = "s", since we already converted our timestamp to seconds while scraping
btc_price["date"] = pd.to_datetime(btc_price["timestamp"], unit="s")
btc_price

,timestamp,close,date
0,1722470400,64643,2024-08-01 00:00:00
1,1722470460,64642,2024-08-01 00:01:00
2,1722470520,64639,2024-08-01 00:02:00
3,1722470580,64663,2024-08-01 00:03:00
4,1722470640,64688,2024-08-01 00:04:00
...,...,...,...
1435,1722556500,65375,2024-08-01 23:55:00
1436,1722556560,65390,2024-08-01 23:56:00
1437,1722556620,65374,2024-08-01 23:57:00
1438,1722556680,65356,2024-08-01 23:58:00


In [17]:
btc_price = btc_price.set_index("date")["close"]
btc_price

date
2024-08-01 00:00:00    64643
2024-08-01 00:01:00    64642
2024-08-01 00:02:00    64639
2024-08-01 00:03:00    64663
2024-08-01 00:04:00    64688
                       ...  
2024-08-01 23:55:00    65375
2024-08-01 23:56:00    65390
2024-08-01 23:57:00    65374
2024-08-01 23:58:00    65356
2024-08-01 23:59:00    65293
Name: close, Length: 1440, dtype: int64

In [23]:
rsi = vbt.RSI.run(btc_price, window=14, short_name="rsi")
entries = rsi.rsi_crossed_below(30)
exits = rsi.rsi_crossed_above(70)

In [ ]:
pf = vbt.Portfolio.from_signals(btc_price, entries, exits)
pf.plot().show()

Optimising variables

In [32]:
num = 10
entry_points = np.linspace(1,45, num=num)
exit_points = np.linspace(55,99, num=num)

entries = rsi.rsi_crossed_below(list(entry_points))
exits = rsi.rsi_crossed_above(list(exit_points))

pf = vbt.Portfolio.from_signals(btc_price, entries, exits)
pf_performance = pf.total_return()

pf_performance

rsi_crossed_below  rsi_crossed_above
1.000000           55.000000            0.000000
5.888889           59.888889            0.000000
10.777778          64.777778            0.000000
15.666667          69.666667           -0.017403
20.555556          74.555556           -0.010620
25.444444          79.444444           -0.011055
30.333333          84.333333            0.002169
35.222222          89.222222           -0.007168
40.111111          94.111111            0.006671
45.000000          99.000000            0.010118
Name: total_return, dtype: float64

Plotting heatmap

In [ ]:
grid = np.array(np.meshgrid(entry_points, exit_points)).T.reshape(-1, 2)

entries = rsi.rsi_crossed_below(list(grid[:, [0]]))
exits = rsi.rsi_crossed_above(list(grid[:, [1]]))

pf = vbt.Portfolio.from_signals(btc_price, entries, exits)
pf_performance = pf.total_return()

pf_performance_matrix = pf_performance.vbt.unstack_to_df(index_levels="rsi_crossed_above", column_levels="rsi_crossed_below")

pf_performance_matrix.vbt.heatmap(xaxis_title="entry", yaxis_title="exit").show()